## Depth-1 Neural Network for Classical Linear Regression

The classical linear regression admits an observed matrix of random variable $X$ and the corresponding vector of random variable $Y$. The model assumes that $\vec{x_i}$ for $i = 1...p$ in $X\in R^{n\times p}$ are iid. normally distributed, and the conditional mean of $Y$ depends on $X$ has a linear relationship with some parameters. Namely,

$$
    E[Y|X=x] = \hat{Y} = Xw + b
$$

where $w$ and $b$ are unknown.

## Loss Function

The common loss function for linear regression is squared $L^2$ Norm which quantifies the distance between two surfaces. 
$$
L(w,b) = \sum_{i\in[n]}(\hat{y}-y)^2 = ||\hat{Y} - Y||^2_2

Note that, unlike most models, linear regression has an analytical solutions:
$$
w^* = (X^TX)^{-1}X^TY
$$

## Minibatch Optimization
Both GD and SGD algorithm have notable drawbacks in practial application. The intermediate approach, minibatch SGD, is often used. Let $B$ be a fixed number of training samples, define the update rule to be:
$$
w_{t+1} = w_t - \frac{\eta}{|B_t|}\sum_{i\in B_t}\partial_{w_t}l^{(i)}(w,b)

Remark: Based on the normality assumption of the data, we can deduce that the Maximum Likelihood Method is equivalent to Minimizing the Squared Loss if we omit the nuissance parameter $\sigma^2$.